In [6]:
import os
import numpy as np
import torch
from moleculekit.molecule import Molecule
import parmed
import nglview as ng
import MDAnalysis as md

import openmm.app as app
from openmm import Platform, LangevinMiddleIntegrator, XmlSerializer
from openmm.unit import picosecond, kelvin
from openmm import unit
import openmm

import mymd

In [11]:
data_path = '/Users/jerrykwan/Downloads/'
pdb_file = os.path.join(data_path, 'aspirin.pdb')
prmtop_file = os.path.join(data_path, 'aspirin.prmtop')
inpcrd_file = os.path.join(data_path, 'aspirin.inpcrd')

cutoff = 9
box = np.array([20., 20., 20.])
T = 300
device = 'cpu'
precision = torch.float
use_centered = True

In [9]:
u = md.Universe(pdb_file)
ng.show_mdanalysis(u, gui=True)

NGLWidget()

In [10]:
mol = mymd.get_molecule(prmtop_file=prmtop_file, pdb_file=pdb_file)

In [12]:
class MyAmberParameterSet(parmed.amber.AmberParameterSet):
    """Inherited from AmberParameterSet to modify from_structure method."""

    @classmethod
    def my_from_structure(cls, struct):
        return super(parmed.amber.AmberParameterSet, cls).from_structure(struct, allow_unequal_duplicates=True)


structure = parmed.load_file(prmtop_file)
try:
    prm = parmed.amber.AmberParameterSet().from_structure(structure)
except:
    print('wrong')
    prm = MyAmberParameterSet.my_from_structure(structure)
print(isinstance(structure, parmed.amber.AmberParm))

a = unit.Quantity((box[0] * unit.angstrom, 0 * unit.angstrom, 0 * unit.angstrom))
b = unit.Quantity((0 * unit.angstrom, box[1] * unit.angstrom, 0 * unit.angstrom))
c = unit.Quantity((0 * unit.angstrom, 0 * unit.angstrom, box[2] * unit.angstrom))
structure.box_vectors = (a, b, c)


def disableDispersionCorrection(system):
    # According to openMM:
    # The long range dispersion correction is primarily useful when running simulations at constant pressure, since it
    # produces a more accurate variation in system energy with respect to volume.
    # So I will disable it to avoid implementing it for now.
    from openmm import NonbondedForce

    for f in system.getForces():
        if isinstance(f, NonbondedForce):
            f.setUseDispersionCorrection(False)


system = structure.createSystem(nonbondedMethod=app.CutoffPeriodic,
                                nonbondedCutoff=cutoff * unit.angstrom,
                                switchDistance=0)
system.setDefaultPeriodicBoxVectors(a, b, c)
disableDispersionCorrection(system)

integrator = openmm.LangevinIntegrator(T * unit.kelvin, 1 / unit.picoseconds, 2 * unit.femtoseconds)
platform = openmm.Platform.getPlatformByName('OpenCL')
properties = dict()

context = openmm.Context(system, integrator, platform, properties)
context.setPositions(mol.coords * unit.angstrom)
energies = parmed.openmm.energy_decomposition(structure, context)
energies

2023-05-15 14:41:23,624 - parmed.structure - INFO - Adding bonds...
2023-05-15 14:41:23,625 - parmed.structure - INFO - Adding angles...
2023-05-15 14:41:23,625 - parmed.structure - INFO - Adding dihedrals...
2023-05-15 14:41:23,625 - parmed.structure - INFO - Adding Ryckaert-Bellemans torsions...
2023-05-15 14:41:23,626 - parmed.structure - INFO - Adding Urey-Bradleys...
2023-05-15 14:41:23,626 - parmed.structure - INFO - Adding improper torsions...
2023-05-15 14:41:23,626 - parmed.structure - INFO - Adding CMAP torsions...
2023-05-15 14:41:23,627 - parmed.structure - INFO - Adding trigonal angle terms...
2023-05-15 14:41:23,627 - parmed.structure - INFO - Adding out-of-plane bends...
2023-05-15 14:41:23,627 - parmed.structure - INFO - Adding pi-torsions...
2023-05-15 14:41:23,627 - parmed.structure - INFO - Adding stretch-bends...
2023-05-15 14:41:23,628 - parmed.structure - INFO - Adding torsion-torsions...
2023-05-15 14:41:23,628 - parmed.structure - INFO - Adding Nonbonded force..

True


{'bond': 3.0544656640484957,
 'angle': 1.706726350474312,
 'dihedral': 5.671557687208711,
 'nonbonded': -88.74841401955365,
 'total': -78.31567651227145}

In [35]:
try:
    ff = mymd.PrmtopMolForceField(mol, prmtop_file, allow_unequal_duplicates=False)
except:
    print('False causes error, use True.')
    ff = mymd.PrmtopMolForceField(mol, prmtop_file, allow_unequal_duplicates=True)

system = mymd.System(mol, ff, cutoff=cutoff)
system.set_periodic_box_manual(box.reshape(3, 1))
integrator = mymd.VelocityVerletIntegrator(2)

simulation = mymd.Simulation(mol, system, integrator, device, precision, use_centered=use_centered)
simulation.set_positions(mol.coords)
simulation.set_velocities_to_temperature(T=T)
simulation.update_potentials_and_forces()

print(simulation.potentials)
print(simulation.potentials_sum)
print(torch.max(torch.abs(simulation.forces)))

{'bonds': 3.0544779300689697, 'angles': 1.7067383527755737, 'dihedrals': 5.644763469696045, 'impropers': 0.026793889701366425, 'lj': 18.051368713378906, 'electrostatics': -95.05093383789062}
-66.56679148226976
tensor(72.0845)


In [18]:
print(mol.element)

['O' 'C' 'C' 'O' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'O' 'O' 'H' 'H' 'H' 'H' 'H'
 'H' 'H' 'H']


In [37]:
xyz = np.load(os.path.join(data_path, 'md17_aspirin.npz'))
print(list(xyz['z']))
print(xyz['R'][0])
print(xyz['E'][:15])
# print(np.max(np.abs(xyz['F'][0])))
np.save(os.path.join(data_path, 'xyz.npy'), xyz['R'][0])

[6, 6, 6, 6, 6, 6, 6, 8, 8, 8, 6, 6, 8, 1, 1, 1, 1, 1, 1, 1, 1]
[[ 2.2393 -0.3791  0.263 ]
 [ 0.8424  1.9231 -0.4249]
 [ 2.8709  0.8456  0.2722]
 [ 2.1751  1.9935 -0.0703]
 [-3.4838  0.4953 -0.0896]
 [ 0.891  -0.4647 -0.0939]
 [ 0.1908  0.6991 -0.4402]
 [-0.9633 -1.8425 -0.4185]
 [-1.6531  0.8889  1.3406]
 [ 0.8857 -2.8883  0.2267]
 [ 0.209  -1.772  -0.1069]
 [-2.0185  0.6853  0.2071]
 [-1.1189  0.6285 -0.7886]
 [ 0.3962 -3.7219  0.2035]
 [ 2.7867 -1.2719  0.5268]
 [ 0.3069  2.8224 -0.6911]
 [ 3.913   0.9108  0.5482]
 [ 2.6781  2.9492 -0.0604]
 [-3.736  -0.5623 -0.012 ]
 [-4.0763  1.0637  0.6273]
 [-3.6988  0.8471 -1.0986]]
[[-406757.59126402]
 [-406757.03429293]
 [-406755.65420705]
 [-406753.83377324]
 [-406752.02145762]
 [-406750.54737373]
 [-406749.51483279]
 [-406748.80367635]
 [-406748.1725602 ]
 [-406747.38329599]
 [-406746.26498811]
 [-406744.73929505]
 [-406742.85736811]
 [-406740.84565851]
 [-406739.05751445]]
